In [0]:
# install ktrain on Google Colab
!pip3 install ktrain

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My Drive

/content/drive/My Drive


In [0]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

In [0]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [0]:
!sudo apt-get install wamerican

In [0]:
1
!cat /usr/share/dict/words > english.txt

In [0]:
def ReadEnglishDictionary():
  with open('english.txt', 'r') as f:
    english_words = f.readlines()
  english_words = set([lmtzr.lemmatize(x.strip().lower().replace('\'s', '')) for x in english_words] )
  english_words = dict.fromkeys(english_words, None)
  return english_words

def AddOurReplacements(cleanup_replace_words):
  cleanup_replace_words["vehicle"] = "car"
  cleanup_replace_words["truck"] = "car"
  cleanup_replace_words["bus"] = "car"
  cleanup_replace_words["honda"] = "car"
  cleanup_replace_words["mazda"] = "car"
  cleanup_replace_words["toyota"] = "car"
  cleanup_replace_words["hyundai"] = "car"
  cleanup_replace_words["suzuki"] = "car"
  cleanup_replace_words["chevrolet"] = "car"
  cleanup_replace_words["buick"] = "car"
  cleanup_replace_words["dodge"] = "car"
  cleanup_replace_words["nissan"] = "car"
  cleanup_replace_words["renault"] = "car"
  cleanup_replace_words["mitsubishi"] = "car"
  cleanup_replace_words["oldsmobile"] = "car"
  cleanup_replace_words["mercedes"] = "car"
  return cleanup_replace_words

In [0]:
import nltk
import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
stop_words = set(nltk.corpus.stopwords.words('english'))
stop_words = set(map(lambda x:x.lower(),stop_words))
# Ignore Not as a Stop Word
stop_words -= {"not"}

lmtzr = WordNetLemmatizer()

english_words = ReadEnglishDictionary()

In [0]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)
  
def lemmatize_stemming(text):
   return  lmtzr.lemmatize(text, get_wordnet_pos(text))

import csv

def ReadCleanupText():
  values = {}
  # github.com/pratikpc/AIHackathon/blob/master/spelling-lemmatized.csv
  with open("lemmatized.csv", "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in reader:
      key = line[0].strip().lower()
      value = line[1].strip().lower()
      values[key] = value
  return values
cleanup_replace_words = ReadCleanupText()
cleanup_replace_words = AddOurReplacements(cleanup_replace_words)

In [0]:
non_eng_sentence = []
def ReplaceToAmericanEnglishWithoutShortCuts(word):
  if word in cleanup_replace_words:
    word = cleanup_replace_words[word]
  return word

def IsEnglishWord(word):
  return (word in english_words)

def ApplyCleanup(sentence):
  # Remove Mentions and URLs
  sentence = re.sub(r"(?:\@|https?\://)\S+", "", sentence)
  split = sentence.split()
  split = [ReplaceToAmericanEnglishWithoutShortCuts(lemmatize_stemming(word.strip().lower())) for word in split if len(word) > 1]
  sentence = ' '.join(split)
  split = sentence.split()
  eng = []
  non_eng = []
  for word in split:
    if word is not None and word not in stop_words and IsEnglishWord(word):
      eng.append(word)
    elif word is not None and word not in stop_words:
      non_eng.append(word)
  sentence = ' '.join(eng)
  non_eng_ind = ' '.join(non_eng)
  non_eng_sentence.append(non_eng_ind)
  return sentence

In [0]:
import pandas as pd

Data = "data.csv"
data = pd.read_csv(Data)
data.drop('Unnamed: 0', axis =1, inplace = True)
data.label = data.label.astype(int)

data = pd.concat([data, enquiry, vehicle], ignore_index=True)
#data.drop('Unnamed: 0', axis =1, inplace = True)


In [0]:
data.head()

,text,label
0,stall put new spark plug,2
1,start know anyone could help ford taurus turn ...,2
2,idle really rough second,2
3,nissan stuck park,2
4,ford speed column stuck first gear,2


In [0]:
data['label'] = data['label'].astype(int)
data['label'].value_counts()

4    1107
0     822
3     200
1     196
2     157
Name: label, dtype: int64

In [0]:
#https://github.com/amaiya/ktrain/blob/master/ktrain/text/data.py
(x_train,  y_train), (x_test, y_test), preproc = ktrain.text.texts_from_df(data, 'text', ['label'], preprocess_mode='bert')

preprocessing train...


preprocessing test...


## STEP 2:  Load the BERT Model and Instantiate a Learner object

In [0]:
# you can disregard the deprecation warnings arising from using Keras 2.2.4 with TensorFlow 1.14.
model = ktrain.text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 400
done.


## STEP 3: Train the Model

We train using one of the three learning rates recommended in the BERT paper: *5e-5*, *3e-5*, or *2e-5*.
Alternatively, the ktrain Learning Rate Finder can be used to find a good learning rate by invoking `learner.lr_find()` and `learner.lr_plot()`, prior to training.
The `learner.fit_onecycle` method employs a [1cycle learning rate policy](https://arxiv.org/pdf/1803.09820.pdf).



In [0]:
# learner.lr_find()
# learner.lr_plot()

In [0]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
  90/2233 [>.............................] - ETA: 3:49 - loss: 0.0845 - acc: 0.9667

We can use the `learner.validate` method to test our model against the validation set.
As we can see, BERT achieves a **96%** accuracy, which is quite a bit higher than the 91% accuracy achieved by SVM in the [scikit-learn tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).

In [0]:
learner.validate(val_data=(x_test, y_test))

              precision    recall  f1-score   support

           0       0.80      0.74      0.77        82
           1       0.48      0.58      0.52        19
           2       0.83      0.68      0.75        22
           3       0.89      0.89      0.89        19
           4       0.87      0.92      0.89       107

    accuracy                           0.81       249
   macro avg       0.78      0.76      0.77       249
weighted avg       0.82      0.81      0.81       249



array([[61,  9,  1,  2,  9],
       [ 5, 11,  1,  0,  2],
       [ 2,  1, 15,  0,  4],
       [ 0,  2,  0, 17,  0],
       [ 8,  0,  1,  0, 98]])

## How to Use Our Trained BERT Model

We can call the `learner.get_predictor` method to obtain a Predictor object capable of making predictions on new raw data.

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [0]:
predictor.get_classes()

['label']

In [0]:
data = [ 'I want to take a test drive',
         'I want to buy a new car',
        'Lot of noise is coming out of engine',
       'I am not satisfied with the service',
       'My car is not working I need assistance']

predictions = predictor.predict(data, return_proba=True)
print(predictions.argmax())

14


In [0]:
import numpy as np
predictions = [np.argmax(x) for x in predictions]
predictions

[1, 0, 4, 1, 2]

The `predictor.save` and `ktrain.load_predictor` methods can be used to save the Predictor object to disk and reload it at a later time to make predictions on new data.

In [0]:
# let's save the predictor for later use
predictor.save('predictor2_77')

In [0]:
# reload the predictor
# reloaded_predictor = ktrain.load_predictor('predictor2_77')

In [0]:
# make a prediction on the same document to verify it still works
# reload_pred = reloaded_predictor.predict(data, return_proba=True)
# reload_pred = [np.argmax(x) for x in reload_pred]
# reload_pred

In [0]:
# data_1 = ['I was thinking of buying a the tata tiago',
#           'I am looking to test drive the tata tiago',
#           'My tata tiago broke down in the middle of the road',
#           'I am satsified with the service i received at the garage',
#           'There is a lot of noise coming out of the engine of my tata tiago'
#       ]
# predict1 = predictor.predict(data_1, return_proba=True)
# predict1 = [np.argmax(x) for x in predict1]
# predict1

In [0]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True)
learners = []
for i in range(5):
  result = next(kf.split(data), None)
  X_train = data['text'].iloc[result[0]]
  X_test = data['text'].iloc[result[1]]
  y_train = data['label'].iloc[result[0]]
  y_test = data['label'].iloc[result[1]]
  (x_train,  y_train), (x_test, y_test), preproc = ktrain.text.texts_from_array(x_train=X_train, y_train=y_train, x_test=X_test, y_test=y_test, preprocess_mode='bert')
  model = ktrain.text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
  learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)
  history = learner.fit_onecycle(2e-5, 5)
  learner.validate(val_data=(x_test, y_test))
  learners.append(learner)
  

preprocessing train...


preprocessing test...


Is Multi-Label? False
maxlen is 400
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
1187/1187 [==============================] - 251s 211ms/step - loss: 1.4019 - acc: 0.4263
Epoch 2/5
1187/1187 [==============================] - 241s 203ms/step - loss: 0.7790 - acc: 0.7245
Epoch 3/5
1187/1187 [==============================] - 240s 202ms/step - loss: 0.4445 - acc: 0.8324
Epoch 4/5
1187/1187 [==============================] - 240s 202ms/step - loss: 0.2286 - acc: 0.9082
Epoch 5/5
1187/1187 [==============================] - 240s 202ms/step - loss: 0.1223 - acc: 0.9545
              precision    recall  f1-score   support

           0       0.66      0.72      0.69        72
           1       0.53      0.49      0.51        47
           2       0.80      0.77      0.78        26
           3       0.93      0.78      0.85        36
           4       0.92      0.95      0.93       116

    accuracy                           0.78       297
   macro avg   

preprocessing test...


Is Multi-Label? False
maxlen is 400
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
1187/1187 [==============================] - 252s 212ms/step - loss: 1.3062 - acc: 0.4777
Epoch 2/5
1187/1187 [==============================] - 240s 202ms/step - loss: 0.7352 - acc: 0.7313
Epoch 3/5
1187/1187 [==============================] - 240s 202ms/step - loss: 0.4008 - acc: 0.8509
Epoch 4/5
1187/1187 [==============================] - 243s 204ms/step - loss: 0.2041 - acc: 0.9183
Epoch 5/5
1187/1187 [==============================] - 244s 206ms/step - loss: 0.1228 - acc: 0.9495
              precision    recall  f1-score   support

           0       0.60      0.70      0.65        54
           1       0.46      0.35      0.40        37
           2       0.59      0.66      0.62        29
           3       0.89      0.76      0.82        41
           4       0.91      0.93      0.92       136

    accuracy                           0.77       297
   macro avg   

preprocessing test...


Is Multi-Label? False
maxlen is 400
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
1187/1187 [==============================] - 257s 217ms/step - loss: 1.2996 - acc: 0.4709
Epoch 2/5
1187/1187 [==============================] - 244s 205ms/step - loss: 0.7607 - acc: 0.7262
Epoch 3/5
 408/1187 [=========>....................] - ETA: 2:40 - loss: 0.4455 - acc: 0.8456